<a href="https://colab.research.google.com/github/narendra974/AIMLOPS_IISC/blob/main/NMT_TRANSFORMERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DATASETS/ENG_DEU_DATA/

/content/drive/MyDrive/DATASETS/ENG_DEU_DATA


In [3]:
!pip install pycld2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycld2: filename=pycld2-0.41-cp310-cp310-linux_x86_64.whl size=9904066 sha256=624a9b9d1f0ef7d67b8f01772e0fdf711fb6fc7984e77c7b8569bcf8dd7b5ea2
  Stored in directory: /root/.cache/pip/wheels/be/81/31/240c89c845e008a93d98542325270007de595bfd356eb0b06c
Successfully built pycld2


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import pycld2 as cld2
import spacy
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import regex as re

In [5]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!python -m spacy download de_core_news_sm
gernlp = spacy.load('de_core_news_sm')
engnlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


2023-12-22 23:27:20.421511: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 34.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [6]:
def removestop(text,stopwords):
  raw = text.split()
  words = [word for word in raw if not word in stopwords]
  cleanwords = ' '.join(words)
  return cleanwords

def tolower(text):
  return text.lower()

def removespecial(text):
  te1 = re.sub("\s+"," ",text)
  te2 = re.sub('\n', '', te1)
  te3 = re.sub('\r', '', te2)
  te4 = re.sub("[0-9]","",te3)
  te5 = re.sub("()@%^&*-_,/\{}[?|$|.|!]","",te4)
  te6 = re.sub(r"[\p{Cc}\p{Cs}]+","",te5)
  te7 = re.sub(r'[^\w\s]','', te6)
  te8 = re.sub("[^a-zA-Z ]","",te7)
  return te7

def removeurl(text):
  return re.sub('https?://\S+|www\.\S+', '', text)

def clean_text(text):
    text = text.lower()
    pattern = re.compile(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?")
    text = re.sub(pattern,' ',text).strip()
    return text

In [7]:
basefolder =  "/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/"
germanfiles = ["commoncrawl_de_en.txt","europarl-v7_de_en.txt","news-commentary-v9_de_en.txt"]
engfiles = ["commoncrawl_en_de.txt","europarl-v7_en_de.txt","news-commentary-v9_en_de.txt"]

In [8]:
def read_files(fileloc, language):
  with open(fileloc,"rb") as f:
    f_lines = f.readlines()
  df = pd.DataFrame(f_lines)
  dfc = df.set_axis([language],axis=1)
  dfc[language] = dfc[language].str.decode("utf-8")
  return dfc

In [9]:
dfappend=pd.DataFrame()
for efile in range(len(germanfiles)):
  germanfilepath = basefolder+germanfiles[efile]
  print(germanfilepath)
  germandff = read_files(germanfilepath,"german")
  engfilepath = basefolder+engfiles[efile]
  engdff = read_files(engfilepath,"english")
  print(germandff.shape)
  print(engdff.shape)
  dfconcat = pd.concat([germandff, engdff],axis="columns")
  dfappend=pd.concat([dfappend, dfconcat])

/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/commoncrawl_de_en.txt
(2399123, 1)
(2399123, 1)
/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/europarl-v7_de_en.txt
(1920209, 1)
(1920209, 1)
/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/news-commentary-v9_de_en.txt
(201288, 1)
(201288, 1)


In [10]:
dfappend.isna().sum()

german     0
english    0
dtype: int64

In [11]:
dfappend.duplicated().sum()

45510

In [12]:
dfappend.drop_duplicates(subset=None, keep='first', inplace=True)
dfappend.shape

(4475110, 2)

In [13]:
german_stop_words = stopwords.words('german')
english_stop_words = stopwords.words('english')

dfappend['english_clean'] = dfappend['english'].apply(lambda x: removestop(x,english_stop_words))
dfappend['german_clean'] = dfappend['german'].apply(lambda x: removestop(x,german_stop_words))
dfappend['german_clean'] = dfappend['german_clean'].apply(lambda x: removespecial(x))
dfappend['english_clean'] = dfappend['english_clean'].apply(lambda x: removespecial(x))

In [14]:
def langdet(x):
  isReliable, textBytesFound, details = cld2.detect(x)
  return(details[0][1])

In [15]:
dfappend['is_eng'] = dfappend['english_clean'].apply(lambda x: langdet(x))
dfappend['is_ger'] = dfappend['german_clean'].apply(lambda x: langdet(x))

In [17]:
display(dfappend["is_ger"].value_counts())

de         4166624
en          127489
un          125032
fr           16352
es            9554
            ...   
hmn              1
ab               1
xx-Goth          1
ta               1
ts               1
Name: is_ger, Length: 132, dtype: int64

In [18]:
display(dfappend["is_eng"].value_counts())

en    4344138
un      73499
de      17085
es       9187
fr       8192
       ...   
ve          1
hi          1
to          1
ky          1
ts          1
Name: is_eng, Length: 127, dtype: int64

In [19]:
dfappendclean = dfappend[(dfappend.is_eng == 'en') & (dfappend.is_ger =='de')]

In [20]:
dfappendclean.shape

(4091605, 6)

In [21]:
display(dfappendclean)

,german,english,english_clean,german_clean,is_eng,is_ger
0,"iron cement ist eine gebrauchs-fertige Paste, ...",iron cement is a ready for use paste which is ...,iron cement ready use paste laid fillet putty ...,iron cement gebrauchsfertige Paste Spachtel Fi...,en,de
1,Nach der Aushärtung schützt iron cement die Ko...,iron cement protects the ingot against the hot...,iron cement protects ingot hot abrasive steel ...,Nach Aushärtung schützt iron cement Kokille he...,en,de
2,feuerfester Reparaturkitt für Feuerungsanlagen...,"a fire restant repair cement for fire places, ...",fire restant repair cement fire places ovens o...,feuerfester Reparaturkitt Feuerungsanlagen Öfe...,en,de
3,Der Bau und die Reparatur der Autostraßen...\n,Construction and repair of highways and...\n,Construction repair highways and,Der Bau Reparatur Autostraßen,en,de
4,die Mitteilungen sollen den geschäftlichen kom...,An announcement must be commercial character.\n,An announcement must commercial character,Mitteilungen sollen geschäftlichen kommerziell...,en,de
...,...,...,...,...,...,...
201283,Das bleibt eine der größten Errungenschaften i...,Their achievement remains one of the greatest ...,Their achievement remains one greatest recent ...,Das bleibt größten Errungenschaften jüngeren G...,en,de
201284,Gleichzeitig scheint sich Zumas revolutionäre ...,"At the same time, Zuma’s revolutionary generat...",At time Zumas revolutionary generation still s...,Gleichzeitig scheint Zumas revolutionäre Gener...,en,de
201285,"In einer Region, wo die älteren Menschen sehr ...","In a region that reveres the elderly, Zuma’s a...",In region reveres elderly Zumas attachment rur...,In Region älteren Menschen verehrt werden Zuma...,en,de
201286,Drei von zehn Südafrikanern sind jünger als 15...,Three in ten South Africans are younger than 1...,Three ten South Africans younger meaning live...,Drei zehn Südafrikanern jünger bedeutet Tag A...,en,de


In [23]:
dfappendclean["engcount"]=dfappendclean['english_clean'].str.split().str.len()

<ipython-input-23-b3f513c107bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfappendclean["engcount"]=dfappendclean['english_clean'].str.split().str.len()
